In [2]:
import numpy as np
import tifffile
import zarr
import dask.array as da
import pyometiff
from contextlib import redirect_stdout

from dask.cache import Cache
cache = Cache(4e9)  # Leverage 4 gigabytes of memory
cache.register()


In [3]:
path = '/fastdata/tmp/SB001-T01-01-HE.ome.tiff'
tif = tifffile.TiffFile(str(path))
if not tif.is_ome:
    raise RuntimeError("only OME TIFF files are accepted.")

ome = pyometiff.OMETIFFReader(fpath=str(path))
ome.omexml_string = tif.ome_metadata  # work-around a bug in pyometiff
with redirect_stdout(None): # to avoid messages about not found keys
    info = ome.parse_metadata(tif.ome_metadata)

# axes: identify singletons and remove from axes annotation
axes = ''.join(
    [a for a in list(info['DimOrder']) if info['Size' + a] > 1]
).lower() # -> something like 'cyx'

# axes of interest
axes_order = {
    'c': list(axes).index('c'), 
    'y': list(axes).index('y'),
    'x': list(axes).index('x'),
    } # just X, Y, C

tif = tifffile.imread(path, aszarr=True)
if not tif.is_multiscales:
    raise RuntimeError("only pyramidal images are accepted.")

if info['PhysicalSizeXUnit'] == 'µm':
    unit_multiplier_x = 1.0 
elif info['PhysicalSizeXUnit'] == 'mm': 
    unit_multiplier_x = 1000.0
else:
    unit_multiplier_x = 1.0
    raise RuntimeWarning('unknown unit for resolution (X)')

if info['PhysicalSizeYUnit'] == 'µm':
    unit_multiplier_y = 1.0 
elif info['PhysicalSizeYUnit'] == 'mm': 
    unit_multiplier_y = 1000.0
else:
    unit_multiplier_y = 1.0
    raise RuntimeWarning('unknown unit for resolution (Y)')

base_mpp_x = unit_multiplier_x * info['PhysicalSizeX']  # in microns per pixel
base_mpp_y = unit_multiplier_y * info['PhysicalSizeY']

pyramid = None
with zarr.open(tif, mode='r') as z:
    n_levels = len(list(z.array_keys()))
    pyramid = [
        da.moveaxis( 
            da.from_zarr(z[i]), [axes_order['y'], axes_order['x'], axes_order['c']], [0, 1, 2] 
        ) 
        for i in range(n_levels)
        ]
metadata = {
#        'rgb': True,
    'channel_axis': 2,
    'contrast_limits': (0, 255),
    'multiscale': True,
}

layer_type = "image"  
print("Pyramid shapes:")
for p in pyramid:
    print(p.shape)

Pyramid shapes:
(170180, 88218, 3)
(85090, 44109, 3)
(42545, 22054, 3)
(21272, 11027, 3)
(10636, 5513, 3)
(5318, 2756, 3)
(2659, 1378, 3)
(1329, 689, 3)
(664, 344, 3)
(332, 172, 3)
(166, 86, 3)


In [4]:
pyramid[0].dtype

dtype('uint8')